In [2]:
!pip -q install streamlit "transformers>=4.43.3" "accelerate>=0.33.0" \
"bitsandbytes>=0.43.1" "peft>=0.11.1" "datasets>=2.20.0" "evaluate>=0.4.2" \
"pandas>=2.2.2" "numpy>=1.26.0" "tqdm>=4.66.4" "sentencepiece>=0.2.0" spacy

# optional for quick tokenization-only stuff
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 107.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import os
# Streamlit doesn't need Metaflow env here; only HF token if you’ll load gated Llama.
if "HF_TOKEN" not in os.environ or not os.environ["HF_TOKEN"]:
    try:
        from getpass import getpass
        os.environ["HF_TOKEN"] = getpass("Paste your HF token (press Enter to skip for BioGPT): ")
    except Exception:
        pass

print("HF_TOKEN set?" , bool(os.environ.get("HF_TOKEN")))


Paste your HF token (press Enter to skip for BioGPT): ··········
HF_TOKEN set? True


In [12]:
%%writefile med_helpers.py
import re, json
from typing import List, Dict, Tuple
import numpy as np

FIELDS = ["Examination","Clinical","Findings","Impression"]

ABBREV_MAP = {
    "wnl": "within normal limits",
    "nl": "normal",
}

def norm_txt(x: str) -> str:
    if not isinstance(x, str):
        return ""
    x = x.strip()
    # remove code fences if the model wrapped JSON
    x = re.sub(r"^```(?:json)?\s*|\s*```$", "", x.strip(), flags=re.I)
    x = x.lower()
    x = re.sub(r"[ \t\r\n]+", " ", x)
    x = re.sub(r"[,:;]+", " ", x)
    x = re.sub(r"\s{2,}", " ", x).strip(" .")
    for k,v in ABBREV_MAP.items():
        x = re.sub(rf"\b{k}\b", v, x)
    return x

def build_prompt(report_text: str, fields=FIELDS) -> str:
    # keep your intent/format; add explicit “start/end with braces” hint
    max_chars = {"Examination":120,"Clinical":160,"Findings":600,"Impression":300}
    quoted_keys = ", ".join([f'"{k}"' for k in fields])

    role = ("You are a precise medical information extraction engine. "
            "Your only job is to extract specific sections from the given radiology report.")
    schema = (
        "Return a single JSON object with exactly these keys and string values:\n"
        f"{quoted_keys}.\n"
        "The JSON must be valid and parseable by a standard JSON parser. "
        "Begin your output with '{' and end with '}'. "
        "No extra keys, no comments, no explanations, no markdown."
    )
    constraints = (
        "Rules:\n"
        "1) Copy or lightly paraphrase from the report; do NOT invent clinical facts.\n"
        "2) If a section is missing or truly unspecified, use an empty string \"\".\n"
        "3) Remove patient identifiers/PHI. Keep clinical terminology intact.\n"
        "4) Keep each field concise and readable. Do not include section headers in values.\n"
        "5) Do not include JSON code fences. Output JSON only.\n"
        "6) Respect length limits (characters, not tokens):\n"
        f"   - Examination: <= {max_chars['Examination']}\n"
        f"   - Clinical:    <= {max_chars['Clinical']}\n"
        f"   - Findings:    <= {max_chars['Findings']}\n"
        f"   - Impression:  <= {max_chars['Impression']}\n"
        "7) If multiple mentions exist for a section, merge succinctly into one value.\n"
        "8) Normalize whitespace: single spaces, no trailing punctuation unless natural.\n"
        "9) Do not include quotes inside values unless they are part of the clinical text."
    )
    few_shot = (
        "Examples:\n"
        "Example 1 (well-structured input):\n"
        "Input:\n"
        "  Exam: Chest X-ray PA and lateral\n"
        "  Clinical indication: Dyspnea and cough for 3 days.\n"
        "  Findings: Cardiomediastinal silhouette normal. No focal consolidation or effusion.\n"
        "  Impression: No acute cardiopulmonary disease.\n"
        "JSON:\n"
        "{\n"
        "  \"Examination\": \"Chest X-ray PA and lateral\",\n"
        "  \"Clinical\": \"Dyspnea and cough for 3 days\",\n"
        "  \"Findings\": \"Cardiomediastinal silhouette normal. No focal consolidation or effusion\",\n"
        "  \"Impression\": \"No acute cardiopulmonary disease\"\n"
        "}\n"
        "\n"
        "Example 2 (messy input, missing Impression):\n"
        "Input:\n"
        "  ... portable ap chest performed at bedside ...\n"
        "  hx: fever; r/o pneumonia. lungs clear; no focal opacity; heart size wnl.\n"
        "JSON:\n"
        "{\n"
        "  \"Examination\": \"Portable AP chest radiograph\",\n"
        "  \"Clinical\": \"Fever; rule out pneumonia\",\n"
        "  \"Findings\": \"Lungs clear; no focal opacity; cardiac size within normal limits\",\n"
        "  \"Impression\": \"\"\n"
        "}\n"
    )
    task = f"Input report:\n{report_text}\n\nNow output ONLY the JSON object:"
    return "\n".join([role, "", schema, "", constraints, "", few_shot, task])

def strip_code_fences(text: str) -> str:
    return re.sub(r"^```(?:json)?\s*|\s*```$", "", text.strip(), flags=re.I)

def safe_json(text: str) -> Dict[str,str]:
    if not isinstance(text, str):
        return {}
    text = strip_code_fences(text)

    # direct
    try:
        obj = json.loads(text)
        if isinstance(obj, dict): return obj
    except Exception:
        pass

    # last {...} block
    blocks = re.findall(r"\{[\s\S]*\}", text)
    for cand in reversed(blocks):
        try:
            obj = json.loads(cand)
            if isinstance(obj, dict): return obj
        except Exception:
            continue

    # lenient key:"value" pairs (no nested quotes)
    out = {}
    for k in FIELDS:
        mm = re.search(rf'"{re.escape(k)}"\s*:\s*"([^"]*)"', text)
        if mm:
            out[k] = mm.group(1)
    return out

# ----------------- Regex fallback extractor (fixed) -----------------
_header_map = {
    "examination": ["examination", "exam", "study", "procedure"],
    "clinical":    ["clinical history", "history", "clinical indication", "indication"],
    "findings":    ["findings", "results"],
    "impression":  ["impression", "conclusion", "opinion", "assessment"],
}

# Build a single pattern capturing any header and its value once.
def _compile_pattern():
    all_headers = []
    for canon, names in _header_map.items():
        for n in names:
            all_headers.append((canon, n))
    # longer names first
    all_headers.sort(key=lambda x: -len(x[1]))
    header_group = "|".join([re.escape(n) for _, n in all_headers])
    # Capture 'header' then 'value' until next header or end. Allow optional ':' or '-' after header.
    pat = rf"(?P<header>(?:{header_group}))\s*[:\-]?\s*(?P<value>.*?)(?=(?:{header_group})\s*[:\-]?|$)"
    return re.compile(pat, flags=re.I|re.S)

_SECTION_RE = _compile_pattern()

# Quick lookup from name -> canonical key
_name_to_canon = {}
for canon, names in _header_map.items():
    for n in names:
        _name_to_canon[n.lower()] = canon

def rule_extract_sections(text: str) -> Dict[str,str]:
    if not isinstance(text, str):
        return {k:"" for k in FIELDS}
    d = {"Examination":"", "Clinical":"", "Findings":"", "Impression":""}
    for m in _SECTION_RE.finditer(text):
        header = (m.group("header") or "").strip().lower()
        value  = (m.group("value")  or "").strip()
        canon = _name_to_canon.get(header)
        if canon:
            key = canon.capitalize()
            val = re.sub(r"\s+", " ", value).strip()
            d[key] = norm_txt(val)
    return d

# ---------- Metrics ----------
def token_f1(pred: str, gold: str) -> Tuple[float,float,float]:
    ptoks = [t for t in re.findall(r"\w+", norm_txt(pred))]
    gtoks = [t for t in re.findall(r"\w+", norm_txt(gold))]
    if len(ptoks) == 0 and len(gtoks) == 0:
        return 1.0, 1.0, 1.0
    if len(ptoks) == 0 or len(gtoks) == 0:
        return 0.0, 0.0, 0.0
    pset, gset = set(ptoks), set(gtoks)
    inter = len(pset & gset)
    prec = inter / max(1, len(pset))
    rec  = inter / max(1, len(gset))
    f1   = (0.0 if prec+rec==0 else 2*prec*rec/(prec+rec))
    return prec, rec, f1


Overwriting med_helpers.py


In [15]:
%%writefile app.py
import os, json, re
import streamlit as st
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import evaluate as hf_eval

from med_helpers import (
    FIELDS, build_prompt, safe_json, norm_txt, token_f1, rule_extract_sections
)

st.set_page_config(page_title="Radiology IE — Prompted LLM", page_icon="🩺", layout="wide")
st.title("🩺 Radiology Information Extraction (Prompt-engineered LLM)")

with st.sidebar:
    st.header("Model & Inference Settings")
    model_choice = st.selectbox(
        "Choose a model",
        [
            "meta-llama/Llama-3.2-1B (gated — requires HF token)",
            "microsoft/BioGPT (ungated — smaller)",
        ],
        index=0
    )
    max_new_tokens = st.slider("Max new tokens", 64, 512, 256, 32)
    do_sample = st.checkbox("Sampling (otherwise greedy)", value=False)
    temperature = st.slider("Temperature", 0.0, 1.5, 0.0, 0.1)
    use_regex_fallback = st.checkbox("Enable regex fallback when JSON is missing/empty", value=True)
    st.caption("Tip: keep greedy decoding (temperature=0) for deterministic JSON.")
    st.markdown("---")
    st.subheader("Evaluation Settings")
    show_debug = st.checkbox("Show raw generations", value=False)

HF_TOKEN = os.environ.get("HF_TOKEN", None)
chosen_model = "meta-llama/Llama-3.2-1B" if "Llama" in model_choice else "microsoft/BioGPT"

@st.cache_resource(show_spinner=True)
def load_model(model_name: str, hf_token: str | None):
    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        quantization_config=bnb,
        use_auth_token=hf_token
    )
    model.eval()
    model.config.use_cache = True
    return tokenizer, model

try:
    tok, model = load_model(chosen_model, HF_TOKEN)
    st.success(f"Loaded model: {chosen_model}")
except Exception as e:
    st.error(f"Failed to load model `{chosen_model}`. If it's gated (Llama), set HF_TOKEN in Colab. Error: {e}")
    st.stop()

def generate_json(report_text: str) -> tuple[dict, str, dict]:
    """Returns (clean_json, raw_text, fallback_used_details)"""
    prompt = build_prompt(report_text)
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            temperature=temperature,
            pad_token_id=tok.pad_token_id,
            eos_token_id=tok.eos_token_id,
        )
    raw = tok.decode(out_ids[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    js = safe_json(raw)

    # Build normalized dict of predictions
    pred = {k: norm_txt(js.get(k, "")) for k in FIELDS}

    fallback_info = {"used": False, "filled_from_regex": []}
    if use_regex_fallback:
        # If any field is empty, attempt regex extraction and fill missing fields
        regex_out = rule_extract_sections(report_text)
        for k in FIELDS:
            if not pred[k] and regex_out.get(k):
                pred[k] = regex_out[k]
                fallback_info["used"] = True
                fallback_info["filled_from_regex"].append(k)

    return pred, raw, fallback_info

# ===== Single report tab =====
tab1, tab2 = st.tabs(["🔎 Single Report", "📊 Batch Evaluation (CSV)"])

with tab1:
    st.subheader("Single Report Extraction")
    report = st.text_area("Paste a radiology report", height=200, placeholder="Enter report text…")
    if st.button("Extract"):
        if not report.strip():
            st.warning("Please paste a report.")
        else:
            pred, raw, fb = generate_json(report)
            colA, colB = st.columns([1,1])
            with colA:
                st.write("**Predicted JSON**")
                st.code(json.dumps(pred, indent=2), language="json")
                if fb["used"]:
                    st.info(f"Regex fallback filled: {', '.join(fb['filled_from_regex'])}")
            with colB:
                st.write("**Structured Output**")
                for f in FIELDS:
                    st.markdown(f"**{f}:** {pred[f] or '—'}")
            if show_debug:
                with st.expander("Raw Generation"):
                    st.code(raw)

with tab2:
    st.subheader("Batch Evaluation")
    st.markdown(
        "Upload a CSV with a `text` column and (optionally) any of the gold columns: "
        "`Examination`, `Clinical`, `Findings`, `Impression`."
    )
    file = st.file_uploader("Choose CSV", type=["csv"])
    if file:
        df = pd.read_csv(file)
        # Rename so the rest of the code still works without modification
        if "ReportText" in df.columns and "text" not in df.columns:
          df.rename(columns={"ReportText": "text"}, inplace=True)

        if "text" not in df.columns:
            st.error("CSV must include a `text` column.")
        else:
            df["text"] = df["text"].astype(str)
            preds = {f"pred_{f}":[] for f in FIELDS}
            raws, fb_cols = [], []
            prog = st.progress(0)
            for i, t in enumerate(df["text"].tolist(), start=1):
                pr, raw, fb = generate_json(t)
                for f in FIELDS:
                    preds[f"pred_{f}"].append(pr[f])
                raws.append(raw)
                fb_cols.append(",".join(fb["filled_from_regex"]) if fb["used"] else "")
                prog.progress(i/len(df))
            for k, v in preds.items():
                df[k] = v
            if show_debug:
                df["raw"] = raws
            df["regex_fallback_filled"] = fb_cols

            rouge = hf_eval.load("rouge")
            per_field = {}
            for f in FIELDS:
                if f in df.columns:
                    gold = df[f].fillna("").apply(norm_txt).tolist()
                    pred = df[f"pred_{f}"].fillna("").apply(norm_txt).tolist()
                    exact = np.mean([p==g for p,g in zip(pred,gold)])
                    prfs = [token_f1(p,g) for p,g in zip(pred,gold)]
                    prec = float(np.mean([x[0] for x in prfs]))
                    rec  = float(np.mean([x[1] for x in prfs]))
                    f1   = float(np.mean([x[2] for x in prfs]))
                    rfs = []
                    for p,g in zip(pred,gold):
                        sc = rouge.compute(predictions=[p], references=[g])
                        rfs.append(sc.get("rougeL", 0.0))
                    rougeL = float(np.mean(rfs))
                    per_field[f] = {
                        "exact_match": float(exact),
                        "token_precision": prec,
                        "token_recall": rec,
                        "token_f1": f1,
                        "rougeL_f1": rougeL
                    }
                else:
                    per_field[f] = "no_gold"

            scored = [f for f,v in per_field.items() if isinstance(v, dict)]
            macro = (
                {
                    "exact_match": float(np.mean([per_field[f]["exact_match"] for f in scored])),
                    "token_precision": float(np.mean([per_field[f]["token_precision"] for f in scored])),
                    "token_recall": float(np.mean([per_field[f]["token_recall"] for f in scored])),
                    "token_f1": float(np.mean([per_field[f]["token_f1"] for f in scored])),
                    "rougeL_f1": float(np.mean([per_field[f]["rougeL_f1"] for f in scored])),
                }
                if scored else "no_gold"
            )

            st.markdown("### Metrics")
            st.json({"per_field": per_field, "macro": macro})

            st.markdown("### Preview (first 50 rows)")
            st.dataframe(df.head(50))

            st.download_button(
                "Download full predictions CSV",
                data=df.to_csv(index=False).encode("utf-8"),
                file_name="predictions.csv",
                mime="text/csv"
            )

st.markdown("---")
with st.expander("Show Prompt Template"):
    st.code(build_prompt("...example report..."))


Overwriting app.py


In [16]:
# --- Clean up any old processes ---
!pkill -f cloudflared >/dev/null 2>&1 || true

# --- 1) Start Streamlit (background) ---
!streamlit run app.py --server.port 8501 --server.headless true &>/content/app.log &

# --- 2) Wait for Streamlit to come up on localhost:8501 ---
import time, urllib.request, urllib.error

def wait_for_streamlit(url="http://127.0.0.1:8501", timeout=120):
    t0 = time.time()
    while time.time() - t0 < timeout:
        try:
            with urllib.request.urlopen(url, timeout=2) as r:
                if r.status == 200:
                    return True
        except Exception:
            pass
        time.sleep(1)
    return False

print("⏳ Waiting for Streamlit to start...")
ok = wait_for_streamlit()
print("Streamlit ready:", ok)
if not ok:
    print("⚠️ Streamlit didn't become ready in time. Dumping last 60 lines of app.log for clues:")
    !tail -n 60 /content/app.log
    raise SystemExit

# --- 3) Download cloudflared if needed ---
import os, subprocess, re, sys

BIN = "/content/cloudflared"
if not os.path.exists(BIN):
    !wget -q -O /content/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
    !chmod +x /content/cloudflared

# --- 4) Launch cloudflared and parse its logs for the public URL ---
LOG = "/content/cf.log"
# Start cloudflared in the background, logging to a file
p = subprocess.Popen([BIN, "tunnel", "--no-autoupdate", "--url", "http://127.0.0.1:8501"],
                     stdout=open(LOG, "w"), stderr=subprocess.STDOUT, text=True)

# Tail the log looking for the URL
public_url = None
pattern = re.compile(r"(https://[a-z0-9\-]+\.trycloudflare\.com)", re.I)

print("⏳ Waiting for cloudflared URL...")
for _ in range(240):  # ~4 minutes
    time.sleep(1)
    try:
        with open(LOG, "r") as f:
            txt = f.read()
            m = pattern.search(txt)
            if m:
                public_url = m.group(1)
                break
    except FileNotFoundError:
        pass

if public_url:
    print("✅ Your Streamlit app is live at:", public_url)
    print("Keep this cell running to keep the tunnel alive.")
else:
    print("❌ Could not detect the tunnel URL.")
    print("Last 80 lines of cloudflared log:")
    !tail -n 80 /content/cf.log
    print("\nLast 60 lines of app.log:")
    !tail -n 60 /content/app.log


^C
⏳ Waiting for Streamlit to start...
Streamlit ready: True
⏳ Waiting for cloudflared URL...
✅ Your Streamlit app is live at: https://notified-lexmark-qui-singles.trycloudflare.com
Keep this cell running to keep the tunnel alive.
